# Tutorial: Utilizing Open Source Language Models with AutoGen

AutoGen is compatible with the OpenAI API library in Python for executing language models. Consequently, it can work with any models employing a similar API without the necessity to modify your AutoGen code.


In this guide, we will utilize the  [modelz-llm](https://github.com/tensorchord/modelz-llm) package to illustrate how to locally serve a model and integrate AutoGen with the served model.


## Instructions

1. First, install the modelz-llm package using the command: ```pip install modelz-llm```
2. Next, initiate the serving of the model; for instance, executing ```modelz-llm -m bigscience/bloomz-560m --device auto``` will locally serve the bloomz-560m small model.
3. Lastly, you will need to modify the `OAI_CONFIG_LIST` file or the `config_list` variable to point to the local model. In this scenario, we are using the default port 8000.


Here, we utilize the Bloomz-560M model, which can run on a CPU for demonstration purposes. It's anticipated that the quality of responses will be low.

## The AutoGen Code

Below, we demonstrate how to configure the `config_list` variable, and subsequently invoke the Completion method.


In [1]:
# Setup autogen with the correct API
import autogen

autogen.oai.ChatCompletion.start_logging()

config_list = [
        {
            'model': 'bloomz-560m-my-small-model',
            'api_key': 'any string here is fine',
            'api_type': 'openai',
            'api_base': "http://localhost:8000",
            'api_version': '2023-03-15-preview'
        }
]


In [2]:
# Perform Completion
question = "Who are you?"
response = autogen.oai.Completion.create(config_list=config_list, prompt=question, temperature=0)
ans = autogen.oai.Completion.extract_text(response)[0]

print("The small model's answer is:", ans)

The small model's answer is:  a human being


## A More Sophisticated Example Featuring Multiple Agents

It's noteworthy that various LLMs can be employed for different agents. In this example, we showcase two agents; one utilizes the bloomz model while the other employs GPT models (as derived from the `OAI_CONFIG_LIST`. It's presumed that you have an environment variable or a local file named "OAI_CONFIG_LIST" set up).

In [3]:
from autogen import AssistantAgent, UserProxyAgent

local_config_list = [
        {
            'model': 'bloomz-560m-my-small-model',
            'api_key': 'any string here is fine',
            'api_type': 'openai',
            'api_base': "http://localhost:8000",
            'api_version': '2023-03-15-preview'
        }
]


gpt_config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-4-32k-v0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
            "gpt",
        }
    }
)

In [4]:
small = AssistantAgent(name="small model",
                       max_consecutive_auto_reply=2,
                       system_message="You should act as a student!",
                       llm_config={
                           "config_list": local_config_list,
                           "temperature": 1,
                       })

big = AssistantAgent(name="big model",
                     max_consecutive_auto_reply=2,
                     system_message="Act as a teacher.",
                     llm_config={
                         "config_list": gpt_config_list,
                         "temperature": 1,
                     })


In [5]:
big.initiate_chat(small, message="Who are you?")


big model (to small model):

Who are you?

--------------------------------------------------------------------------------
small model (to big model):

 Who are you?

--------------------------------------------------------------------------------
big model (to small model):

I am an AI language model, programmed to provide information, answer questions, and assist with various tasks. As a teacher, I can help explain various topics and concepts. What would you like to learn about today?

--------------------------------------------------------------------------------
small model (to big model):

 What is the topic of the day? User

--------------------------------------------------------------------------------
big model (to small model):

As an AI teacher, I don't have a predetermined topic of the day. However, I'm more than happy to discuss or teach any topic you'd like to learn about or explore. Please let me know your area of interest, and we can dive into it right away.

--------